In [ ]:
!pip install split_folders

In [ ]:
import split_folders

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import metrics

from sklearn.utils import class_weight
from collections import Counter

import matplotlib.pyplot as plt

import os
from os import listdir
from os.path import isfile, join
print(os.path)

In [ ]:
os.makedirs('output')
os.makedirs('output/train')
os.makedirs('output/val')

In [ ]:
os.makedirs('output/test')

In [ ]:
!ls ../input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images

In [ ]:
img_loc = '../input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/'

split_folders.ratio(img_loc, output='output', seed=1, ratio=(0.1,0.8, 0.1))

In [ ]:
!ls output

In [ ]:
train_loc = 'output/train/'
val_loc = 'output/val/'
test_loc = 'output/test/'


In [ ]:
img_loc= 'output/train'

In [ ]:
trdata = ImageDataGenerator()
testdata = trdata.flow_from_directory(directory=train_loc, target_size=(224,224))
tsdata = ImageDataGenerator()
traindata = tsdata.flow_from_directory(directory=val_loc, target_size=(224,224))

In [ ]:
vldata = ImageDataGenerator()
valdata = vldata.flow_from_directory(directory=test_loc, target_size=(224,224))

In [ ]:
vgg16 = VGG16(weights='imagenet')
vgg16.summary()

x  = vgg16.get_layer('fc2').output
prediction = Dense(5, activation='softmax', name='predictions')(x)

model = Model(inputs=vgg16.input, outputs=prediction)

In [ ]:
for layer in model.layers:
    layer.trainable = False

for layer in model.layers[-16:]:
    layer.trainable = True
    print("Layer '%s' is trainable" % layer.name)  

In [ ]:
from keras.optimizers import SGD
#opt = optimizers.sgd(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer='Adadelta', loss=categorical_crossentropy, 
              metrics=['accuracy'])
model.summary()

In [ ]:
checkpoint = ModelCheckpoint("vgg16_diabetes.h5", monitor='val_accuracy', verbose=1, 
                             save_best_only=True, save_weights_only=False, mode='auto')
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')

In [ ]:
counter = Counter(traindata.classes)                       
max_val = float(max(counter.values()))   
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}
class_weights

In [ ]:
hist = model.fit(traindata, steps_per_epoch=traindata.samples//traindata.batch_size, validation_data=valdata, 
                 class_weight=class_weights, validation_steps=valdata.samples//valdata.batch_size, 
                 epochs=100,callbacks=[checkpoint,early])

In [ ]:
plt.plot(hist.history['loss'], label='train')
plt.plot(hist.history['val_loss'], label='val')
plt.title('VGG16: Loss and Validation Loss (0.0001 = Adam LR)')
plt.legend();
plt.show()

plt.plot(hist.history['accuracy'], label='train')
plt.plot(hist.history['val_accuracy'], label='val')
plt.title('VGG16: Accuracy and Validation Accuracy (0.0001 = Adam LR)')
plt.legend();
plt.show()

In [ ]:

import itertools  
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
import numpy as np
#model = Model(inputs=vgg16.input, outputs=prediction)
#model = Model(inputs=base_model.input, outputs= model(vgg16.output))
#model = Model(input= base_model.input, output= model(base_model.output))
Y_pred = model.predict_generator(testdata)
y_pred = np.argmax(Y_pred, axis=1)


In [ ]:
for i in range(len(y_pred)):
    print(y_pred)

In [ ]:
#Compute confusion matrix
from sklearn.metrics import classification_report, confusion_matrix
target_names = ['No_DR','Mild','Moderate','Severe','Proliferate_DR']
cnf_matrix = confusion_matrix(testdata.classes, y_pred)
np.set_printoptions(precision=2)
print(cnf_matrix.shape)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=target_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=target_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

In [ ]:
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
#from keras.models import Sequential #initialized the nural network#create cnn for 2d here image is 2d ,but video is 3d
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense #layer fully connected ann 
 

In [ ]:
y_real = []
y_pred =[]

In [ ]:

import cv2
import glob
import numpy as np




for filename in glob.glob('output//test//Mild//*.png'): #assuming gif
    print(filename)
    img=cv2.imread(filename)
    #print(im)
    #img = cv2.resize(img,(224,244))
    img = np.reshape(img,[1,224,224,3])
    classes = model.predict(img)
    maxi = -1
    for i in range(5):
        if classes[0][i]> maxi :
            index = i
            maxi = classes[0][i]
    y_real.append(0)
    y_pred.append(index)
    # convert the probabilities to class labels
   
# print the classification
   


    

In [ ]:
for filename in glob.glob('output//test//Moderate//*.png'): #assuming gif
    print(filename)
    img=cv2.imread(filename)
    #print(im)
    #img = cv2.resize(img,(224,244))
    img = np.reshape(img,[1,224,224,3])
    classes = model.predict(img)
    maxi = -1
    for i in range(5):
        if classes[0][i]> maxi :
            index = i
            maxi = classes[0][i]
    y_real.append(1)
    y_pred.append(index)
    # convert the probabilities to class labels
   
# print the classification

In [ ]:
for filename in glob.glob('output//test//No_DR//*.png'): #assuming gif
    print(filename)
    img=cv2.imread(filename)
    #print(im)
    #img = cv2.resize(img,(224,244))
    img = np.reshape(img,[1,224,224,3])
    classes = model.predict(img)
    maxi = -1
    for i in range(5):
        if classes[0][i]> maxi :
            index = i
            maxi = classes[0][i]
    y_real.append(2)
    y_pred.append(index)
    # convert the probabilities to class labels
   
# print the classification

In [ ]:
for filename in glob.glob('output//test//Proliferate_DR//*.png'): #assuming gif
    print(filename)
    img=cv2.imread(filename)
    #print(im)
    #img = cv2.resize(img,(224,244))
    img = np.reshape(img,[1,224,224,3])
    classes = model.predict(img)
    maxi = -1
    for i in range(5):
        if classes[0][i]> maxi :
            index = i
            maxi = classes[0][i]
    y_real.append(3)
    y_pred.append(index)
    # convert the probabilities to class labels
   
# print the classification

In [ ]:
for filename in glob.glob('output//test//Severe//*.png'): #assuming gif
    print(filename)
    img=cv2.imread(filename)
    #print(im)
    #img = cv2.resize(img,(224,244))
    img = np.reshape(img,[1,224,224,3])
    classes = model.predict(img)
    maxi = -1
    for i in range(5):
        if classes[0][i]> maxi :
            index = i
            maxi = classes[0][i]
    y_real.append(4)
    y_pred.append(index)
    # convert the probabilities to class labels
   
# print the classification

In [ ]:
from sklearn.metrics import accuracy_score,precision_score
accuracy = accuracy_score(y_real, y_pred)

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

In [ ]:
print(accuracy*100)
print(y_real)
print(y_pred)

In [ ]:
#precesion.........


# precision tp / (tp + fp)
precision = precision_score(y_real, y_pred,average='macro')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_real, y_pred,average='macro')
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_real, y_pred,average='macro')
print('F1 score: %f' % f1)
